In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    
    p_weight = items_weights['weight'].tolist()
    
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=p_weight)
    
    return recs.tolist()

In [6]:
%%time

items_weights = data[['item_id', 'sales_value']]
items_weights.groupby('item_id')['sales_value'].sum().reset_index()
items_weights = items_weights[items_weights['sales_value'] >= 1]
items_weights = items_weights.copy()
items_weights['log_sales_value'] = np.log(items_weights['sales_value'])
items_weights = items_weights.copy()
items_weights['weight'] = items_weights['log_sales_value'] / items_weights['log_sales_value'].sum()
items_weights = items_weights[['item_id', 'weight']]

Wall time: 665 ms


In [7]:
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head()

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1076306, 5569230, 9553397, 874972, 7467082]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1135552, 977499, 6534178, 1117250, 12301128]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[8291322, 973630, 1013433, 7168026, 957736]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[961756, 1105301, 968215, 1107446, 1112405]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[852065, 1177591, 948832, 871754, 1126375]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [8]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [103]:
# your_code
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [104]:
ranrec_precision_at_k = 0
poprec_precision_at_k = 0
itemitem_precision_at_k = 0
cosine_precision_at_k = 0
tfidf_precision_at_k = 0
own_purchases_precision_at_k = 0

In [137]:
k = 5
for i in result.index:
    ranrec_precision_at_k += precision_at_k([number for number in result.iloc[i][2].split()], [number for number in result.iloc[i][1].split()], k=k)
    poprec_precision_at_k += precision_at_k([number for number in result.iloc[i][3].split()], [number for number in result.iloc[i][1].split()], k=k)
    itemitem_precision_at_k += precision_at_k([number for number in result.iloc[i][4].split()], [number for number in result.iloc[i][1].split()], k=k)
    cosine_precision_at_k += precision_at_k([number for number in result.iloc[i][5].split()], [number for number in result.iloc[i][1].split()], k=k)
    tfidf_precision_at_k += precision_at_k([number for number in result.iloc[i][6].split()], [number for number in result.iloc[i][1].split()], k=k)
    own_purchases_precision_at_k += precision_at_k([number for number in result.iloc[i][7].split()], [number for number in result.iloc[i][1].split()], k=k)

In [138]:
print(f'precision ranrec_precision@{k} = {ranrec_precision_at_k / result.shape[0]:.4%}')
print(f'precision poprec_precision@{k} = {poprec_precision_at_k / result.shape[0]:.4%}')
print(f'precision itemitem@{k} = {itemitem_precision_at_k / result.shape[0]:.4%}')
print(f'precision cosine@{k} = {cosine_precision_at_k / result.shape[0]:.4%}')
print(f'precision tfidf@{k} = {tfidf_precision_at_k / result.shape[0]:.4%}')
print(f'precision own_purchases@{k} = {own_purchases_precision_at_k / result.shape[0]:.4%}')

precision ranrec_precision@5 = 0.0490%
precision poprec_precision@5 = 0.4114%
precision itemitem@5 = 0.0196%
precision cosine@5 = 0.0000%
precision tfidf@5 = 0.0000%
precision own_purchases@5 = 0.1371%


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [9]:
# your_code